## This part of the project was done in local PC

**Loading the dev and train data**

In [1]:
import json

# File paths
dev_file_path = r'D:\Rajesh\Rajesh\Personal\AISanDiego\520-NLP\Project\OriginalData\StanfordQA\dev-v1.1.json'
train_file_path = r'D:\Rajesh\Rajesh\Personal\AISanDiego\520-NLP\Project\OriginalData\StanfordQA\train-v1.1.json'

# Load dev data
with open(dev_file_path, 'r', encoding='utf-8') as dev_file:
    dev_squad_data = json.load(dev_file)

# Load train data
with open(train_file_path, 'r', encoding='utf-8') as train_file:
    train_squad_data = json.load(train_file)

# Print the loaded data (optional)
print(f"Sample from dev-v1.1.json: {dev_squad_data['data'][0]['paragraphs'][0]['qas'][0]}")
print(f"Sample from train-v1.1.json: {train_squad_data['data'][0]['paragraphs'][0]['qas'][0]}")

Sample from dev-v1.1.json: {'answers': [{'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}], 'question': 'Which NFL team represented the AFC at Super Bowl 50?', 'id': '56be4db0acb8001400a502ec'}
Sample from train-v1.1.json: {'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}


In [2]:
# Define a function to extract question-answer pairs from the dataset.
def extract_qna_pairs_for_bert(squad_data):
    qna_pairs = []
    for article in squad_data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qas in paragraph['qas']:
                question = qas['question']
                if qas.get('is_impossible', False):
                    answer = "I don't know."
                    answer_start = -1  # No answer available
                else:
                    # Take the first answer
                    answer = qas['answers'][0]['text'] if qas['answers'] else "No answer available."
                    answer_start = qas['answers'][0]['answer_start']
                qna_pairs.append((context, question, answer, answer_start))
    return qna_pairs

# Step 5: Extract question-answer pairs from both train and dev datasets
qna_pairs_train = extract_qna_pairs_for_bert(train_squad_data)
qna_pairs_dev = extract_qna_pairs_for_bert(dev_squad_data)

# Displaying a sample of the extracted data
print(f"Sample from extracted training data: {qna_pairs_train[0]}")
print(f"Sample from extracted dev data: {qna_pairs_dev[0]}")


Sample from extracted training data: ('Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'Saint Bernadette Soubirous', 515)
Sample from extracted dev data: ('Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 20

In [3]:
import re

# Function to clean text (removes unnecessary whitespace, lowercasing, etc.)
def clean_text(text):
    text = text.lower()  # Lowercase the text (since we're using an uncased model)
    text = text.replace('\n', ' ').replace('\r', ' ')  # Remove newline and carriage returns
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Apply cleaning to the extracted question-answer pairs before tokenization
def clean_qna_pairs(qna_pairs):
    cleaned_qna_pairs = []
    for context, question, answer, answer_start in qna_pairs:
        cleaned_context = clean_text(context)
        cleaned_question = clean_text(question)
        cleaned_answer = clean_text(answer)
        cleaned_qna_pairs.append((cleaned_context, cleaned_question, cleaned_answer, answer_start))
    return cleaned_qna_pairs

# Step 6 (Revised): Clean the extracted train and dev data before tokenization
cleaned_qna_pairs_train = clean_qna_pairs(qna_pairs_train)
cleaned_qna_pairs_dev = clean_qna_pairs(qna_pairs_dev)

# Displaying a sample of cleaned data
print(f"Cleaned sample from training data: {cleaned_qna_pairs_train[0]}")


Cleaned sample from training data: ('architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend "venite ad me omnes". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome), is a simple, modern stone statue of mary.', 'to whom did the virgin mary allegedly appear in 1858 in lourdes france?', 'saint bernadette soubirous', 515)


In [8]:
from transformers import BertTokenizerFast

# Load the pre-trained BERT fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Function to tokenize the cleaned question-answer pairs and find the token positions of the answer
def tokenize_qna_pairs(qna_pairs, tokenizer, max_length=128):
    tokenized_data = []

    for context, question, answer, answer_start in qna_pairs:
        # Tokenize the question and context together
        encoding = tokenizer.encode_plus(
            question,
            context,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_offsets_mapping=True,  # Now this will work with the fast tokenizer
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze().tolist()  # Convert to list
        attention_mask = encoding['attention_mask'].squeeze().tolist()  # Convert to list
        token_type_ids = encoding['token_type_ids'].squeeze().tolist()  # Convert to list
        offset_mapping = encoding['offset_mapping'].squeeze().tolist()  # Convert to list

        # Initialize start and end positions of the answer
        start_position = 0
        end_position = 0

        # Find the token positions corresponding to the answer if it's not unanswerable
        if answer_start != -1:
            for idx, (start, end) in enumerate(offset_mapping):
                if start <= answer_start < end:
                    start_position = idx
                if start < answer_start + len(answer) <= end:
                    end_position = idx
                    break

        # Append tokenized data
        tokenized_data.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'start_position': start_position,
            'end_position': end_position
        })

    return tokenized_data

# Tokenize the cleaned question-answer pairs
tokenized_train = tokenize_qna_pairs(cleaned_qna_pairs_train, tokenizer)
tokenized_dev = tokenize_qna_pairs(cleaned_qna_pairs_dev, tokenizer)

# Display a sample of the tokenized data
print(f"Sample tokenized data from training set: {tokenized_train[0]}")


Sample tokenized data from training set: {'input_ids': [101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008

In [4]:
import pickle

# Save tokenized data to files using pickle
with open('tokenized_train.pkl', 'wb') as f:
    pickle.dump(tokenized_train, f)

with open('tokenized_dev.pkl', 'wb') as f:
    pickle.dump(tokenized_dev, f)

print("Pre-processed data saved successfully")


Pre-processed data saved successfully!


### The files tokenized_train.pkl and tokenized_dev.pkl were uploaded to the Gooogle Drive

### This part of the project was done in Google Colabs Pro

In [11]:
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the load path from your Google Drive
load_path_train = '/content/drive/My Drive/SQuAD_datasets/tokenized_train.pkl'
load_path_dev = '/content/drive/My Drive/SQuAD_datasets/tokenized_dev.pkl'

# Load the tokenized training and dev datasets
with open(load_path_train, 'rb') as f:
    tokenized_train = pickle.load(f)

with open(load_path_dev, 'rb') as f:
    tokenized_dev = pickle.load(f)

print("Tokenized data loaded successfully from Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tokenized data loaded successfully from Google Drive.


In [12]:
import torch
from torch.utils.data import Dataset

# Custom Dataset class for Question Answering
class QADataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        # Return the input IDs, attention mask, token type IDs, start and end positions as a dictionary
        item = self.tokenized_data[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'token_type_ids': torch.tensor(item['token_type_ids']),
            'start_positions': torch.tensor(item['start_position']),
            'end_positions': torch.tensor(item['end_position'])
        }

# Step 2: Create Dataset objects for train and dev sets
train_dataset = QADataset(tokenized_train)
dev_dataset = QADataset(tokenized_dev)

# Display a sample from the training dataset
print(train_dataset[0])


{'input_ids': tensor([  101,  2000,  3183,  2106,  1996,  6261,  2984,  9382,  3711,  1999,
         8517,  1999, 10223, 26371,  2605,  1029,   102,  6549,  2135,  1010,
         1996,  2082,  2038,  1037,  3234,  2839,  1012, 10234,  1996,  2364,
         2311,  1005,  1055,  2751,  8514,  2003,  1037,  3585,  6231,  1997,
         1996,  6261,  2984,  1012,  3202,  1999,  2392,  1997,  1996,  2364,
         2311,  1998,  5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,
         4828,  2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,
         2310,  3490,  2618,  4748,  2033, 18168,  5267,  1000,  1012,  2279,
         2000,  1996,  2364,  2311,  2003,  1996, 13546,  1997,  1996,  6730,
         2540,  1012,  3202,  2369,  1996, 13546,  2003,  1996, 24665, 23052,
         1010,  1037, 14042,  2173,  1997,  7083,  1998,  9185,  1012,  2009,
         2003,  1037, 15059,  1997,  1996, 24665, 23052,  2012, 10223, 26371,
         1010,  2605,  2073,  1996,  6261,  2984, 

In [13]:
from torch.utils.data import DataLoader

# Create DataLoaders for the training and dev datasets
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)

# Display a batch of data (for demonstration purposes)
for batch in train_loader:
    print(batch)
    break


{'input_ids': tensor([[ 101, 2054, 2001,  ...,    0,    0,    0],
        [ 101, 2029, 2028,  ...,    0,    0,    0],
        [ 101, 2055, 2129,  ...,    0,    0,    0],
        ...,
        [ 101, 2043, 2064,  ...,    0,    0,    0],
        [ 101, 2054, 4654,  ...,    0,    0,    0],
        [ 101, 2006, 2029,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'start_positions': tensor([ 58,  10,  23,  62,   7,  85, 109,  55]), 'end_positions': tensor([ 59,  11,  23,  63,  11,  91, 113,  56])}


In [14]:
from transformers import BertForQuestionAnswering, AdamW, get_scheduler
import torch
from tqdm import tqdm  # Progress bar

# Load the pre-trained BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_epochs = 3

# Set up the learning rate scheduler
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Set device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Training loop with printing average loss every 100 batches
model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Train the model
    total_train_loss = 0
    batch_losses = []  # To track the losses of the last 100 batches
    progress_bar = tqdm(train_loader, desc="Training")

    for batch_idx, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}  # Move data to the device
        outputs = model(**batch)  # Forward pass
        loss = outputs.loss
        total_train_loss += loss.item()
        batch_losses.append(loss.item())

        optimizer.zero_grad()  # Clear gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update parameters
        lr_scheduler.step()  # Update learning rate

        # Print the average loss for the last 100 batches
        if (batch_idx + 1) % 100 == 0:
            avg_last_100_batches = sum(batch_losses[-100:]) / len(batch_losses[-100:])
            print(f"Average loss of the last 100 batches (batch {batch_idx + 1}): {avg_last_100_batches}")

        # Update progress bar with the current loss
        progress_bar.set_postfix({"loss": loss.item()})

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Average Training Loss for Epoch {epoch + 1}: {avg_train_loss}")

    # Validation loop
    model.eval()
    total_val_loss = 0
    progress_bar = tqdm(dev_loader, desc="Validating")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)  # Forward pass (without computing gradients)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(dev_loader)
    print(f"Validation Loss for Epoch {epoch + 1}: {avg_val_loss}")

    # Set model back to training mode
    model.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Training:   1%|          | 101/10950 [00:18<30:32,  5.92it/s, loss=2.99]

Average loss of the last 100 batches (batch 100): 4.231133337020874


...

Average loss of the last 100 batches (batch 10900): 0.625988946184516


Training: 100%|██████████| 10950/10950 [31:08<00:00,  5.86it/s, loss=0.36]


Average Training Loss for Epoch 3: 0.6020535657558267


Validating: 100%|██████████| 1322/1322 [01:11<00:00, 18.45it/s]

Validation Loss for Epoch 3: 1.3187899800967116


In [15]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory where you want to save the model
model_save_path = '/content/drive/My Drive/SQuAD_datasets/saved_model'

# Create the directory if it doesn't exist
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Save the trained model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")


Model and tokenizer saved to /content/drive/My Drive/SQuAD_datasets/saved_model


In [17]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the saved model and tokenizer from Google Drive
model = BertForQuestionAnswering.from_pretrained(model_save_path)
tokenizer = BertTokenizerFast.from_pretrained(model_save_path)

print("Model and tokenizer loaded successfully.")


Model and tokenizer loaded successfully.


In [19]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Load the saved model and tokenizer from Google Drive
model_save_path = '/content/drive/My Drive/SQuAD_datasets/saved_model'
model = BertForQuestionAnswering.from_pretrained(model_save_path)
tokenizer = BertTokenizerFast.from_pretrained(model_save_path)

# Set the model to evaluation mode
model.eval()

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [20]:
def answer_question(question, context):
    # Tokenize input
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512).to(device)

    # Generate output
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the most likely beginning and end of the answer span
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores
    answer_start = torch.argmax(answer_start_scores)  # Get the most likely start of the answer
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of the answer

    # Convert the tokens to text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer


In [21]:
# Example context and question
context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season.
The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10
to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."""

question = "Who won Super Bowl 50?"

# Get the answer
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: Who won Super Bowl 50?
Answer: denver broncos


In [22]:
# New example context and question (completely different from training or dev data)
context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel,
whose company designed and built the tower. Constructed from 1887 to 1889 as the entrance arch for the 1889 World's Fair,
it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France
and one of the most recognizable structures in the world.
"""

question = "Who designed the Eiffel Tower?"

# Get the answer from the model
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: Who designed the Eiffel Tower?
Answer: gustave eiffel


In [23]:
question = "What is Eiffel Tower made up of?"

# Get the answer from the model
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is Eiffel Tower made up of?
Answer: wrought - iron lattice tower


In [24]:
question = "How long it took to build the Eiffel Tower?"

# Get the answer from the model
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: How long it took to build the Eiffel Tower?
Answer: from 1887 to 1889


In [25]:
question = "Why was the construction of Eiffel Tower critisized initally?"

# Get the answer from the model
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Why was the construction of Eiffel Tower critisized initally?
Answer: it was initially criticized by some of france ' s leading artists and intellectuals for its design


In [27]:
import shutil

# Zip the saved model directory
shutil.make_archive('/content/saved_model', 'zip', '/content/drive/My Drive/SQuAD_datasets/saved_model')

# Download the zip file to your PC
from google.colab import files
files.download('/content/saved_model.zip')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### After downloading the model in Local Laptop, the model was saved to the working directory and zip extracted.

### This part was executed in the local laptop

In [40]:
import sys
print(sys.path)


['C:\\Users\\CZ0068\\torch', 'C:\\Users\\CZ0068\\torch', 'C:\\Users\\CZ0068\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311, C:\\Users\\CZ0068\\torch', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\CZ0068\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages', 'C:\\Users\\CZ0068\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32', 'C:\\Users\\CZ0068\\AppD

In [39]:
import sys
print(sys.executable)


C:\Users\CZ0068\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [38]:
import sys
sys.path.insert(0, 'C:\\Users\\CZ0068\\torch')

import transformers
import torch
import numpy as np

print(transformers.__version__)
print(torch.__version__)
print(np.__version__)


4.45.2
2.4.1+cpu
1.24.4


In [31]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizerFast

# Path to the extracted model on your PC
model_path = 'D:/Rajesh/Rajesh/Personal/AISanDiego/520-NLP/Project/saved_model/'

# Load the model and the fast version of the tokenizer
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

# Define a function to answer questions
def answer_question(question, context):
    # Encode the inputs with offset mappings to preserve context token positions
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", add_special_tokens=True, return_offsets_mapping=True)
    
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    offsets = inputs["offset_mapping"][0]  # Offset mapping for token positions
    
    # Get the answer using the model
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

    # Find the start and end of the answer
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Get the token offsets for the answer (start and end positions)
    start_char = offsets[answer_start][0]
    end_char = offsets[answer_end - 1][1]

    # Extract the answer from the original context using the offsets
    answer = context[start_char:end_char]
    
    # Capitalize the first letter of the answer if needed
    if answer:
        answer = answer[0].upper() + answer[1:]

    return answer

# Test with a new question and context
context = ("Transformers are a type of deep learning model introduced in the paper 'Attention is All You Need' by Vaswani et al. "
           "in 2017. Unlike traditional recurrent neural networks (RNNs), Transformers rely entirely on self-attention mechanisms "
           "to capture the relationships between different parts of a sequence. This allows Transformers to process data in parallel, "
           "making them much faster and more efficient for tasks like natural language processing. Transformers are the foundation for "
           "many state-of-the-art models, such as BERT, GPT, and T5. They are used in a variety of applications, including machine translation, "
           "text generation, and question answering.")



question = "Name some models that are based on the Transformer architecture"
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


question = "What mechanism does the Transformer rely on"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "Why are Transformers faster than traditional RNNs?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")


question = "What happens if selef-attention mechanism capture the relationship between different parts of a sequence ?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "State some applications of Transformers?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "Who introduced self Attention?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

Question: Name some models that are based on the Transformer architecture
Answer: BERT, GPT, and T5
Question: What mechanism does the Transformer rely on
Answer: Self-attention mechanisms
Question: Why are Transformers faster than traditional RNNs?
Answer: Self-attention mechanisms
Question: What happens if selef-attention mechanism capture the relationship between different parts of a sequence ?
Answer: Self-attention mechanisms to capture the relationships between different parts of a sequence. This allows Transformers to process data in parallel
Question: State some applications of Transformers?
Answer: Machine translation, text generation, and question answering
Question: Who introduced self Attention?
Answer: Vaswani et al.


In [37]:
def answer_question(question, context):
    # Tokenize input
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

    # Generate output
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the most likely beginning and end of the answer span
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores
    answer_start = torch.argmax(answer_start_scores)  # Get the most likely start of the answer
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of the answer

    # Convert the tokens to text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    answer=answer[0].upper()+ answer[1:len(answer)]

    return answer

# Test with a new question and context
context = ("Transformers are a type of deep learning model introduced in the paper 'Attention is All You Need' by Vaswani et al. "
           "in 2017. Unlike traditional recurrent neural networks (RNNs), Transformers rely entirely on self-attention mechanisms "
           "to capture the relationships between different parts of a sequence. This allows Transformers to process data in parallel, "
           "making them much faster and more efficient for tasks like natural language processing. Transformers are the foundation for "
           "many state-of-the-art models, such as BERT, GPT, and T5. They are used in a variety of applications, including machine translation, "
           "text generation, and question answering.")
question = "Name some models that are based on the Transformer architecture"
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


question = "What mechanism does the Transformer rely on"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "Why are Transformers faster than traditional RNNs?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")


question = "What happens if selef-attention mechanism capture the relationship between different parts of a sequence ?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "State some applications of Transformers?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

question = "Who introduced self Attention?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")



Question: Name some models that are based on the Transformer architecture
Answer: Bert, gpt, and t5
Question: What mechanism does the Transformer rely on
Answer: Self - attention mechanisms
Question: Why are Transformers faster than traditional RNNs?
Answer: Transformers rely entirely on self - attention mechanisms to capture the relationships between different parts of a sequence
Question: What happens if selef-attention mechanism capture the relationship between different parts of a sequence ?
Answer: Allows transformers to process data in parallel
Question: State some applications of Transformers?
Answer: Machine translation, text generation, and question answering
Question: Who introduced self Attention?
Answer: Vaswani et al.


In [45]:
# Test with a new question and context
context = ("Artificial intelligence (AI) is a field of computer science that aims to create machines that can perform tasks that would normally require human intelligence."
           "These tasks include problem-solving, learning, planning, natural language understanding, and perception. The term was first coined by John McCarthy in 1956 at "
           "the Dartmouth Conference, which is considered the founding event of AI as a field. Early efforts in AI involved symbolic AI, where knowledge was explicitly coded"
           "into machines, but progress was slow due to the complexity of human cognition. In the 1990s and 2000s, a major shift occurred with the rise of machine learning,"
            "a subset of AI that enables computers to learn from data. Instead of being explicitly programmed, machine learning algorithms, especially neural networks, became" 
            "capable of automatically improving their performance on tasks. Deep learning, a branch of machine learning involving multi-layered neural networks, has enabled"
            "breakthroughs in fields like image recognition, speech processing, and game playing.Today, AI is applied in numerous industries including healthcare, finance,"
            " and autonomous driving. While AI has made tremendous progress, it also raises ethical concerns such as bias in algorithms, job displacement, and the need for" 
            "regulations to ensure AI systems are fair and transparent. Despite these challenges, AI continues to evolve rapidly, with researchers aiming to achieve artificial" 
            "general intelligence (AGI), a level of AI that can understand and learn any intellectual task that a human being can.")

question = "What are some of the key developments in artificial intelligence from its early days to the present?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are some of the key developments in artificial intelligence from its early days to the present?
Answer: Problem-solving, learning, planning, natural language understanding, and perception. The term was first coined by John McCarthy in 1956 at the Dartmouth Conference, which is considered the founding event of AI as a field. Early efforts in AI involved symbolic AI, where knowledge was explicitly codedinto machines, but progress was slow due to the complexity of human cognition. In the 1990s and 2000s, a major shift occurred with the rise of machine learning,a subset of AI that enables computers to learn from data. Instead of being explicitly programmed, machine learning algorithms


In [48]:
question = "What are some ethical concerns related to artificial intelligence?"
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are some ethical concerns related to artificial intelligence?
Answer: Bias in algorithms, job displacement, and the need forregulations to ensure AI systems are fair and transparent
